## 인터파크 - 베스트 셀러 

In [128]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

- Step 1) 사이트에 접속하고 HTML 데이터를 가져와 파싱

In [129]:
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001#'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [130]:
print(res.text[:1000])




<!DOCTYPE html>
<html lang="ko">
<head>

 

    <title>인터파크 도서</title>
    <meta http-equiv="Content-Type" content="text/html; charset=euc-kr">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta name="keywords" content="인터파크도서, 인터넷 서점, eBook, 비스킷탭, 음반, DVD, 중고책, 중고책서점, 신간도서, 국내도서, 외국도서, 추천도서">
    <meta name="description" content="문학은 인터파크 도서!">
    <meta http-equiv="Cache-Control" content="no-cache"/>
    <meta http-equiv="Expires" content="0"/>
    <meta http-equiv="Pragma" content="no-cache"/>
    <link rel="shortcut icon" href="http://qi-b.qoo10cdn.com/interpark.ico">
    <meta http-equiv="imagetoolbar" content="no"/>
    <meta name="naver-site-verification" content="295287bf1753708aabe3586a6cbb4b39761ff03b" />
    <script type="text/javascript">
        var isFancyAlert = false;    // IMFS에서는 msgAlert 기능을 비활성화시킴
        var imfsSSLDomain = "https://book.interpark.com"; // com.interpark.util.web.ServletUtils.IMFS_URL_SSL 값을 사용하였으나,


- Step 2) 원하는 데이터 찾기

In [131]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

- Step 3) 하나를 선택해서 원하는 정보 추출

In [132]:
li = lis[0]

In [133]:
# 책 제목
title = li.select_one('.itemName').get_text().strip()
title

'도시와 그 불확실한 벽'

In [134]:
# 저자 
author = li.select_one('.author').get_text().strip()
author

'무라카미 하루키(村上春樹)'

In [135]:
# 출판사
company = li.select_one('.company').get_text().strip()
company

'문학동네'

In [136]:
# 가격 
price = li.select_one('.price > em').get_text().strip()
price = int(price.replace(',',''))
price

17550

In [137]:
img = li.select_one('.cover > a > img')
img


In [138]:
# 순위
"""
<div class="rankNumber digit2">
    <span class="rankBtn_ctrl rkNum_B06 "></span>
</div>

<div class="rankNumber digit2">
    <span class="rankBtn_ctrl rkNum_M01 "></span>
    <span class="rankBtn_ctrl rkNum_M00 "></span>
</div>
"""

'\n<div class="rankNumber digit2">\n    <span class="rankBtn_ctrl rkNum_B06 "></span>\n</div>\n\n<div class="rankNumber digit2">\n    <span class="rankBtn_ctrl rkNum_M01 "></span>\n    <span class="rankBtn_ctrl rkNum_M00 "></span>\n</div>\n'

In [139]:
rank_data = lis[6].select('.rankBtn_ctrl')
rank_data[0]['class']

['rankBtn_ctrl', 'rkNum_B07']

In [140]:
if len(rank_data) == 1:
   rank = rank_data [0]['class'][1][-1]
else:
   rank = rank_data [0]['class'][1][-1] +  rank_data [1]['class'][1][-1]

- Step 4) 반복문을 사용하여 페이지 내에 있는 데이터 가져오기

In [141]:
data = []
for li in lis:
    rank_data = li.select('.rankBtn_ctrl')
    if len(rank_data) == 1:
         rank = rank_data [0]['class'][1][-1]
    else:
        rank = rank_data [0]['class'][1][-1] +  rank_data [1]['class'][1][-1]
        img = li.select_one('.cover > a > img')
    title = li.select_one('.itemName').get_text().strip()
    author = li.select_one('.author').get_text().strip()
    company = li.select_one('.company').get_text().strip()
    price = li.select_one('.price > em').get_text().strip()
    price = int(price.replace(',',''))
    data.append({'순위':rank, '표지':img, '제목':title, '저자': author, '출판사':company, '가격':price})

- Step 5) 오류가 발생하면 오류를 해결하고 2, 3, 4 과정을 반복

- Step 6) 모든 페이지에 대해 반복하기

- Step 7) 데이터프레임으로 만들기 

In [142]:
df = pd.DataFrame(data)
df

,순위,표지,제목,저자,출판사,가격
0,1,None,도시와 그 불확실한 벽,무라카미 하루키(村上春樹),문학동네,17550
1,2,None,디케의 눈물,조국,다산북스,16920
2,3,None,오늘도 나아가는 중입니다,조민,참새책방,15120
3,4,None,퓨처 셀프,벤저민 하디/최은아 역,상상스퀘어,17820
4,5,None,트렌드 코리아 2024,"김난도, 추예린, 전다현, 전미영, 최지혜",미래의창,17100
5,6,None,생각이 너무 많은 어른들을 위한 심리학(10만 부 기념 스페셜 에디션),김혜남,메이븐,16020
6,7,None,일론 머스크,월터 아이작슨,21세기북스,34200
7,7,None,1%를 읽는 힘,메르,토네이도,19800
8,9,None,흔한남매 14,"흔한남매, 백난도/유난희 그림/흔한컴퍼니 감수",미래엔아이세움,13050
9,9,None,한석준의 말하기 수업,한석준,인플루엔셜,15120
